# PycWB Tutorial

## PycWB Version

In [ ]:
import pycwb

print(pycwb.__version__)

## 1. Run your first example

First, we download the example user parameter file

In [ ]:
user_parameters = """
analysis: "2G"
cfg_search: "r"

optim: False

###### network configuration ######
ifo: ["L1","H1"]
refIFO: "L1"

inRate: 16384
# lags
lagSize: 1
lagStep: 1.
lagOff: 0
lagMax: 0

# superlags
slagSize: 0 # number of super lags (simulation=1) - if slagSize=0 -> Standard Segments
slagMin: 0
slagMax: 0
slagOff: 0

# job
segLen: 1200
segMLS: 600
segTHR: 200
segEdge: 10

# frequency
fLow: 16.
fHigh: 1024.


levelR: 3
l_low: 4 # low frequency resolution level		// std (sthr = 2)
l_high: 10 # high frequency resolution level	// std (sthr = 8)

wdmXTalk: "wdmXTalk/OverlapCatalog16-1024.bin"

healpix: 7

###### cWB production thresholds & regulators ######

bpp: 0.001
subnet: 0.5
subcut: 0.0
netRHO: 5.5
# cedRHO: 5.0
netCC: 0.5
Acore: 1.7
Tgap: 0.2
Fgap: 128.0
delta: 0.5
cfg_gamma: -1.0
LOUD: 300

pattern: 5

iwindow: 30

nSky: 196608

# simulation
# simulation: None
nfactor: 1
"""

with open('user_parameters.yaml', 'w') as fp:
    fp.write(user_parameters)

In [ ]:
import os
import shutil
import pycwb
from pycwb.config import Config
from pycwb.modules.logger import logger_init


temp_dir_for_plots = './plots/'

if os.path.exists(temp_dir_for_plots):
    shutil.rmtree(temp_dir_for_plots)
os.makedirs(temp_dir_for_plots)


logger_init()

config = Config('./user_parameters.yaml')

generate injected data for each detector with given parameters in config

In [ ]:
from pycwb.modules.read_data import read_from_catalog, read_from_online
from gwpy.timeseries import TimeSeries

import requests
from gwosc.locate import get_urls
t0 = 1126259462.4

data = []
for ifo in config.ifo:
  url = get_urls(ifo, t0, t0)[-1]

  print('Downloading: ' , url)
  fn = os.path.basename(url)
  with open(fn,'wb') as strainfile:
      straindata = requests.get(url)
      strainfile.write(straindata.content)

  strain = TimeSeries.read(fn,format='hdf5.gwosc')
  d = strain.crop(t0-150, t0+150)
  d_resampled = d.resample(2048)
  data.append(d_resampled)

apply data conditioning to the data

In [ ]:
from pycwb.modules.data_conditioning import data_conditioning
from pycwb.modules.plot import plot_spectrogram

strains, nRMS = data_conditioning(config, data)

# plot the spectrogram for L1
plt = plot_spectrogram(strains[0], gwpy_plot=True)
ax = plt.gca()
ax.set_ylim(15,1024)
plt.savefig(temp_dir_for_plots + f'spectrogram.png')
plt.close()
strains, nRMS

calculate coherence

In [ ]:
from pycwb.modules.coherence import coherence

# calculate coherence
fragment_clusters = coherence(config, strains, nRMS)

In [ ]:
# %matplotlib inline
# from gwpy.spectrogram import Spectrogram
# for fragment_cluster in fragment_clusters:
#   for cluster in fragment_cluster.clusters:
#       merged_map, start, dt, df = cluster.get_sparse_map("likelihood")

#       plt = Spectrogram(merged_map, t0=start, dt=dt, f0=0, df=df).plot()
#       plt.colorbar()

supercluster

In [ ]:
from pycwb.modules.super_cluster import supercluster
from pycwb.types.network import Network

network = Network(config, strains, nRMS)

pwc_list = supercluster(config, network, fragment_clusters, strains)

In [ ]:
# %matplotlib inline
from gwpy.spectrogram import Spectrogram

i=0
for cluster in pwc_list[0].clusters:
    merged_map, start, dt, df = cluster.get_sparse_map("likelihood")
    i+=1
    plt = Spectrogram(merged_map, t0=start, dt=dt, f0=0, df=df).plot()
    plt.colorbar()
    plt.savefig(temp_dir_for_plots + f'sparse_map_{str(i)}_.png')
plt.close()

Likelihood

In [ ]:
from pycwb.modules.likelihood import likelihood

events, clusters, skymap_statistics = likelihood(config, network, pwc_list)

plot statistics

In [ ]:
#%matplotlib inline
from pycwb.modules.plot import plot_event_on_spectrogram

plt = plot_event_on_spectrogram(strains[0], events)
plt.savefig(temp_dir_for_plots + f'event_on_specreogram_{str(i)}_.png')
plt.close()

In [ ]:
#%matplotlib inline
from gwpy.spectrogram import Spectrogram
i=0
for cluster in clusters:
    merged_map, start, dt, df = cluster.get_sparse_map("likelihood")
    i+=1
    plt = Spectrogram(merged_map, t0=start, dt=dt, f0=0, df=df).plot()
    plt.colorbar()
    plt.savefig(temp_dir_for_plots + f'likelihood_{str(i)}_.png')
plt.close()

In [ ]:
#%matplotlib inline
from gwpy.spectrogram import Spectrogram
i=0
for cluster in clusters:
    merged_map, start, dt, df = cluster.get_sparse_map("null")
    i+=1
    plt = Spectrogram(merged_map, t0=start, dt=dt, f0=0, df=df).plot()
    plt.colorbar()
    plt.savefig(temp_dir_for_plots + f'null_{str(i)}_.png')
plt.close()

In [ ]:
from pycwb.modules.reconstruction import get_network_MRA_wave
from pycwb.modules.plot.waveform import plot_reconstructed_waveforms
from matplotlib import pyplot as plt

event = events[0]
cluster = clusters[0]
reconstructed_waves = get_network_MRA_wave(config, cluster, config.rateANA, config.nIFO, config.TDRate,
                                               'signal', 0, True)
for reconstructed_wave in reconstructed_waves:
  plt.plot(reconstructed_wave.sample_times, reconstructed_wave.data)
plt.xlim((event.left[0], event.left[0] + event.stop[0] - event.start[0]))
plt.savefig(temp_dir_for_plots + f'reconstructed_waveforms_{str(i)}_.png')
plt.close()

In [ ]:
events